# HW 5. Пусть бот заговорит. Задание DaNetQA из Russian SuperGLUE

## Часть с телеграмм-ботом

### Работа с моделью DaNETQA

Ссылка на файнтьюнинг моделей: https://github.com/korolevt/otus_NLP_2023/tree/main/hw5/hw5_DaNetQA.ipynb

In [85]:
#!pip install transformers

In [87]:
import torch

from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_scheduler

In [93]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [94]:
# Для загрузки модели
tokenizer = AutoTokenizer.from_pretrained("ruBert-DaNetQA")
model = AutoModelForSequenceClassification.from_pretrained("ruBert-DaNetQA")
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [98]:
def get_reply(text1, text2):
  with torch.no_grad():
    input_msg = tokenizer(text1, text2, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    input_msg = input_msg.to(device)
    outputs = model(**input_msg)[0]
    output = outputs.argmax(axis=1).detach().cpu().numpy()
    probabilities = torch.softmax(outputs, dim=-1).detach().cpu().numpy()
    print(text1, output, probabilities.max())
    return 'Да' if output.item() > 0.5 else 'Нет'

In [99]:
get_reply("Осталась ли библия неизменной с момента создания?", 
'''10 сентября 1750 года Синод доложил императрице, что перевод готов для печати. 
18 декабря 1751 года Елизаветинская Библия вышла из печати. Все изменения, внесённые при исправлении перевода, 
были оговорены, примечания к тексту составили отдельный том, практически равный по объёму тексту самой Библии. 
Первый тираж быстро разошёлся, и в 1756 году вышло его второе издание с дополнительными примечаниями на полях и гравюрами, 
в котором иеромонах Гедеон  исправил ошибки и опечатки первого издания. В дальнейшем Русская церковь продолжила использовать 
в богослужебной практике Елизаветинскую Библию, внеся в неё лишь некоторые несущественные изменения.''')

Осталась ли библия неизменной с момента создания? [0] 0.8368864


'Нет'

### Телеграм-бот

In [ ]:
### https://t.me/OtusHW5QABot - ссылка на бота

In [4]:
#!pip install telebot

In [100]:
import telebot
from telebot import types

import json

In [101]:
#! pip install pyTelegramBotAPI

In [102]:
with open('./config.json', 'r') as f:
    json_config = json.load(f)

In [103]:
TOKEN = json_config['tg_bot_token']

bot = telebot.TeleBot(TOKEN, num_threads=4)

In [104]:
QA = {}

In [105]:
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id,"Проверка DaNetQA из Russian SuperGLUE!")
    msg = bot.send_message(message.chat.id,"Введите текст:")
    bot.register_next_step_handler(msg, question)

In [106]:
def question(message):
    QA['text'] = message.text
    print('text',QA['text'])
    msg = bot.send_message(message.chat.id, "Введите вопрос:")
    bot.register_next_step_handler(msg, answer)

In [107]:
def answer(message):
    QA['question'] = message.text
    print('text',QA['text'])
    print('question',QA['question'])
    
    msg = bot.send_message(message.chat.id, "Ответ:") 
    reply = get_reply(QA['question'],QA['text'])
    msg = bot.send_message(message.chat.id, reply) 
    msg = bot.send_message(message.chat.id, "Для повтора нажмите /start") 
    

In [ ]:
bot.infinity_polling()

## Выводы

Была взяты данные для задачи DANetQA с бенчмарка Russian SuperGLUE (https://russiansuperglue.com/tasks/)

Была зафайнтьюнета модель ruBERT-base (7 эпох)
    
Результаты оценки модели ruBERT (0.681 - для задачи DANetQA)::  
![Результаты оценки модели ruBERT](ruBert-SuperGlue.jpg)    

Также была проведена проверка небольшой модели cointegrated/rubert-tiny2 (2-я версия)  

В модели увеличена максимальная входная длина до 2048 токенов, поэтому текст в модель поместился целеком (без обрезания)
Для 10 эпох, результаты оказались близки к модели ruBert-base.

Результаты оценки модели ruBERT-tiny2 (0.66 - для задачи DANetQA):
![Результаты оценки модели ruBERT-tiny2](ruBert-tiny2-SuperGlue.jpg)    

Для бота была взята модель ruBert-base и использовалась библиотека telebot для написания телеграмм-бота.

Результаты работы бота (на тестовых данных):
![Результаты работы бота](bot.jpg) 
